In [180]:
import torch

import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [221]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

device
torch.cuda.set_device(0) 

In [269]:
df = pd.read_csv('/home/enoch/Documents/data/league_win_predictor_v3.csv')

df['Rank'] = df['Rank'].map(lambda x: x.rstrip('[IV]+'))

def rank_to_num(val):
    
    if val == 'IRON':
        return 1
    elif val == 'BRONZE':
        return 2
    elif val == 'SILVER':
        return 3
    elif val == 'GOLD':
        return 4
    elif val == 'DIAMOND':
        return 5
    elif val == 'PLATINUM':
        return 6
    
def num_to_rank(val):
    
    if val == 1:
        return 'IRON'
    elif val == 2:
        return 'BRONZE'
    elif val == 3:
        return 'SILVER'
    elif val == 4:
        return 'GOLD'
    elif val == 5:
        return 'DIAMOND'
    elif val == 6:
        return 'PLATINUM'
    
def set_not_winner(val):
    if val == 1:
        return 0
    else:
        return 1

df['Rank'] = df['Rank'].map(lambda row: rank_to_num(row))
df['!Winner'] = df['Winner'].map(lambda row: set_not_winner(row))

df = df.drop('match_id', axis=1)

In [258]:
mask = np.random.rand(len(df)) < 0.9

trainset = df[mask]
testset = df[~mask]

X_columns = [col for col in df.columns if col != 'Winner' and col != '!Winner'] + ['Winner']#, '!Winner']

trainset_X = trainset[X_columns]
testset_X = testset[X_columns]

trainloader = torch.utils.data.DataLoader(trainset_X.values, shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset_X.values, shuffle=False, num_workers=2)
testset_X.head()

,B1MP,B1Total,B1Recent,B1WR,B2MP,B2Total,B2Recent,B2WR,B3MP,B3Total,...,R4MP,R4Total,R4Recent,R4WR,R5MP,R5Total,R5Recent,R5WR,Rank,Winner
47,13041,4,0,0.000000,85990,3,0,66.666667,43032,36,...,31719,44,16,65.116279,8454,4,0,0.000000,2,0
50,55702,76,14,46.511628,270262,39,12,0.000000,11589,4,...,8655,8,0,66.666667,7368,3,3,0.000000,2,0
61,41395,24,0,50.000000,10179,18,0,44.444444,74966,41,...,31184,2,2,0.000000,47697,8,1,57.142857,3,0
80,54936,23,0,56.521739,5538,1,0,0.000000,78185,37,...,30459,36,4,55.555556,80562,106,8,52.830189,4,1
87,15535,7,7,0.000000,15489,19,0,25.000000,350838,29,...,11925,25,7,0.000000,71827,135,17,37.837838,2,0


In [259]:
class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
#         self.lin0 = nn.Linear(3, 43)
        self.lin1 = nn.Linear(41, 200)
        self.lin2 = nn.Linear(200, 300)
        self.lin3 = nn.Linear(300, 150)
        self.lin4 = nn.Linear(150, 50)
        self.lin5 = nn.Linear(50, 1)
    
    def forward(self, x):

#         x = F.relu(self.lin0(x))
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = F.relu(self.lin4(x))
        x = self.lin5(x)
        return x
    
    #This function takes an input and predicts the class, (0 or 1)        
    def predict(self,x):
        #Apply softmax to output. 
        pred = F.softmax(self.forward(x))
        ans = []
        #Pick the class with maximum weight
        for t in pred:
            if t[0]>t[1]:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)

In [260]:
net = Net()
net = net.float()#.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [261]:
epochs = 10
losses = []
size = len(trainset_X)
for epoch in range(epochs):
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
    
        datum, winner = data[0][:-1], data[0][-1:]

        optimizer.zero_grad()
        
        y_pred = net.forward(datum.float())#.to(device))
        
        loss = criterion(y_pred,winner.float())#.to(device))
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / size}')


[1, 100] loss: 14679061.28222514
[1, 200] loss: 14684917.155679697
[1, 300] loss: 14686130.5304352
[1, 400] loss: 14686239.149451034
[1, 500] loss: 14687453.590819653
[2, 100] loss: 606.5706056146308
[2, 200] loss: 606.6690057241852
[2, 300] loss: 606.7784799124803
[2, 400] loss: 606.8600726338889
[2, 500] loss: 606.9601465828407
[3, 100] loss: 0.07590487093884052
[3, 200] loss: 0.17395481669663984
[3, 300] loss: 0.2830050497232835
[3, 400] loss: 0.3642575759031922
[3, 500] loss: 0.46389176384755715
[4, 100] loss: 0.07554313182893048
[4, 200] loss: 0.17310361087956103
[4, 300] loss: 0.28158770597614935
[4, 400] loss: 0.3624125144107811
[4, 500] loss: 0.46151398989699866
[5, 100] loss: 0.07512892355747629
[5, 200] loss: 0.17214654572829466
[5, 300] loss: 0.28000651977838775
[5, 400] loss: 0.36033948141706684
[5, 500] loss: 0.45880583238163447
[6, 100] loss: 0.07460932872265169
[6, 200] loss: 0.17092180033967186
[6, 300] loss: 0.2779507876194949
[6, 400] loss: 0.3576321288475977
[6, 500]

In [262]:
dataiter = iter(testloader)
val = dataiter.next()
datum, winner = val[0][:-1], val[0][-1:]

In [268]:
predicted = net(datum.float())

print(f'predicted: {predicted.item()} => expected: {winner.item()}')

predicted: 0.04486396908760071 => expected: 0.0


In [289]:
correct = np.zeros(6)
total = np.zeros(6)

with torch.no_grad():
    for data in testloader:
        datum, winner = data[0][:-1], data[0][-1:]
        output = net(datum.float())
        
        rank = int((datum[-1:]).item() - 1)
        total[rank] += 1
        if round(output.item()) == winner:
            correct[rank] += 1
print(f'correct: {correct} total: {total}')
ratios = correct / total

print(ratios)

correct: [ 1. 14. 11.  4.  1.  2.] total: [ 1. 21. 18.  8.  1.  5.]
[1.         0.66666667 0.61111111 0.5        1.         0.4       ]
